# Carga del modelo

In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import numpy as np


C:\Users\Carla\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
model = gensim.models.Word2Vec.load("./doc2vec_wikipedia_es_pvdbow")
model.docvecs.vectors_docs.shape
model.most_similar(positive=['seguros'],topn=10)


C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('aseguradora', 0.6253476738929749),
 ('reaseguros', 0.5982040166854858),
 ('aseguradoras', 0.5916717052459717),
 ('rga', 0.5637242794036865),
 ('bancarios', 0.5485397577285767),
 ('asegurador', 0.5484205484390259),
 ('bancaria', 0.5317405462265015),
 ('crédit', 0.5307000875473022),
 ('reaseguradoras', 0.5265746116638184),
 ('banca', 0.5258691906929016)]

# Carga de Data

In [3]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
preguntas = bdtesis.PreguntasSBS


In [4]:
#a minusculas
data = []
for x in bdtesis.PreguntasSBS.find():
    data.append(x['Pregunta'])
datamin =[]
for x in (data):
    datamin.append(x.lower()) 


In [5]:
#limpieza de puntuación
import re
import nltk
import string
cadpun = string.punctuation + '¿'
datapunc = []
for x in datamin:
    datapunc.append(x.translate(str.maketrans('', '', cadpun)))


In [6]:
#retirar stopwords
from nltk.corpus import stopwords
stops = stopwords.words('spanish')
dataclean = []
for x in datapunc:
    palabras = x.split()
    resultwords  = [palabra for palabra in palabras if palabra.lower() not in stops]
    dataclean.append(' '.join(resultwords))
    

In [7]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(dataclean)]

In [8]:
datamatrix = []
for x in dataclean:
    palabras = x.split()
    datamatrix.append(palabras)
datamatrix

[['sbs',
  'definirá',
  'codificaciones',
  'activos',
  'emitidos',
  'serie',
  'públicos'],
 ['cuántos', 'días', 'sbs', 'confirme', 'códigos', 'inscritos'],
 ['empresa',
  'podrá',
  'usar',
  'código',
  'propio',
  'interno',
  'recibir',
  'comunicación',
  'sbs'],
 ['sucede', 'tasa', 'renta', 'fija'],
 ['debe', 'colocarse', 'aplica', 'tasa', 'variable'],
 ['emisores',
  'vehículo',
  'emitir',
  'deuda',
  'domiciliada',
  'país',
  'ventajas',
  'tributarias',
  'considera',
  'país',
  'constituye',
  'vehículo',
  'país',
  'riesgo',
  'empresa'],
 ['emisores',
  'vehículo',
  'emitir',
  'deuda',
  'domiciliada',
  'país',
  'ventajas',
  'tributarias',
  'considera',
  'vehículo',
  'emisor',
  'empresa'],
 ['establecerán',
  'códigos',
  'entidades',
  'localesextranjeras',
  'menor',
  'fortaleza',
  'financiera',
  '“a”'],
 ['emisores',
  'reportan',
  'smv',
  'prima',
  'grupo',
  'económico',
  'figura',
  'página',
  'web',
  'entidad'],
 ['cuándo',
  'trate',
  'co

In [14]:
vectoresPF = []
for pregunta in datamatrix:
    vector = model.infer_vector(pregunta)
    vectoresPF.append(vector)
len(vectoresPF)


151

In [13]:
#GUARDADO EN BD, SE CREA UNA COLUMNA NUEVA EN LA COLECCIÓN
cont = 0
for x in bdtesis.PreguntasSBS.find():
    bdtesis.PreguntasSBS.update({"_id": x['_id']}, {"$set": {"vector": vectoresPF[cont].tolist()}})
    cont = cont +1

C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.
